In [1]:
import numpy as np
import pandas as pd
import random as rand
import torch
from torch.utils.data import Dataset
from torchvision.transforms import *

np.random.seed(37)
rand.seed(37)

class GaussianDataset(Dataset):
    def __init__(self, device, loc=0, scale=1, clazz=0, N=1_000):
        self.__device = device
        self.__clazz = clazz
        self.__X = np.random.normal(loc=loc, scale=scale, size=N).reshape(-1, 1)

    def __len__(self):
        return self.__X.shape[0]

    def __getitem__(self, idx):
        item = self.__X[idx,:]
        item = torch.tensor(item, device=self.__device).double()
        
        return item, self.__clazz

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

dataset = GaussianDataset(device=device)

for i in range(10):
    print(i, type(dataset[i]),
          type(dataset[i][0]),
          type(dataset[i][1]),
          dataset[i][1],
          dataset[i][0])

cuda
0 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([-0.0545], device='cuda:0', dtype=torch.float64)
1 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([0.6743], device='cuda:0', dtype=torch.float64)
2 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([0.3466], device='cuda:0', dtype=torch.float64)
3 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([-1.3003], device='cuda:0', dtype=torch.float64)
4 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([1.5185], device='cuda:0', dtype=torch.float64)
5 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([0.9898], device='cuda:0', dtype=torch.float64)
6 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([0.2777], device='cuda:0', dtype=torch.float64)
7 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([-0.4486], device='cuda:0', dtype=torch.float64)
8 <class 'tuple'> <class 'torch.Tensor'> <class 'int'> 0 tensor([0.9620], device='cuda:0', dtype

In [2]:
from torchvision import datasets
from torchvision import transforms

class AE(torch.nn.Module):
    def __init__(self, input_size):
        super().__init__()
          
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(input_size, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 9)
        )
          
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(9, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, input_size)
        )
  
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [3]:
model = AE(input_size=1).double().to(device)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-8)

In [4]:
epochs = 20
outputs = []

for epoch in range(epochs):
    losses = []
    
    for (item, _) in dataset:  
        optimizer.zero_grad()
        
        reconstructed = model(item)
        loss = loss_function(reconstructed, item)
        
        loss.backward()
        
        optimizer.step()
        
        losses.append(loss.detach().cpu().numpy().item())
    
    losses = np.array(losses)
    
    outputs.append({
        'epoch': epoch + 1,
        'loss': losses.mean()
    })

In [5]:
outputs

[{'epoch': 0, 'loss': 0.09523458393174913},
 {'epoch': 1, 'loss': 0.018831867539226145},
 {'epoch': 2, 'loss': 0.0003579292241994011},
 {'epoch': 3, 'loss': 0.02285746374549311},
 {'epoch': 4, 'loss': 0.002193471568789237},
 {'epoch': 5, 'loss': 0.006369732873779139},
 {'epoch': 6, 'loss': 0.006959465555705261},
 {'epoch': 7, 'loss': 0.0054271628551490194},
 {'epoch': 8, 'loss': 0.005842258403331565},
 {'epoch': 9, 'loss': 0.007888257921544838},
 {'epoch': 10, 'loss': 0.004061566742882887},
 {'epoch': 11, 'loss': 0.004239698714537946},
 {'epoch': 12, 'loss': 0.004314447164598894},
 {'epoch': 13, 'loss': 0.0031328218710224604},
 {'epoch': 14, 'loss': 0.007189506953647517},
 {'epoch': 15, 'loss': 0.003018659323039308},
 {'epoch': 16, 'loss': 0.005159008403002595},
 {'epoch': 17, 'loss': 0.002787532322056358},
 {'epoch': 18, 'loss': 0.0025876928968187867},
 {'epoch': 19, 'loss': 0.0018173679436896616}]

In [6]:
for r in range(10):
    print(dataset[r][0].cpu(), model(dataset[r][0]).detach().cpu())

tensor([-0.0545], dtype=torch.float64) tensor([-0.0707], dtype=torch.float64)
tensor([0.6743], dtype=torch.float64) tensor([0.6595], dtype=torch.float64)
tensor([0.3466], dtype=torch.float64) tensor([0.3325], dtype=torch.float64)
tensor([-1.3003], dtype=torch.float64) tensor([-1.3089], dtype=torch.float64)
tensor([1.5185], dtype=torch.float64) tensor([1.5454], dtype=torch.float64)
tensor([0.9898], dtype=torch.float64) tensor([1.0038], dtype=torch.float64)
tensor([0.2777], dtype=torch.float64) tensor([0.2649], dtype=torch.float64)
tensor([-0.4486], dtype=torch.float64) tensor([-0.4463], dtype=torch.float64)
tensor([0.9620], dtype=torch.float64) tensor([0.9731], dtype=torch.float64)
tensor([-0.8276], dtype=torch.float64) tensor([-0.8340], dtype=torch.float64)


In [7]:
for item, _ in dataset:
    print(item.shape)
    print(item)
    break

torch.Size([1])
tensor([-0.0545], device='cuda:0', dtype=torch.float64)
